<a href="https://colab.research.google.com/github/StrideEric/Analisis_de_datos_Informatorio_2025/blob/master/An%C3%A1lisis_de_datos_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Datos

## ETL (Extracción, Transformación y Carga)

### 1. Extracción (Extract)

In [2]:
from google.colab import drive
import pandas as pd
import chardet

drive.mount('/content/drive', force_remount=True)

ruta_archivo = '/content/drive/MyDrive/Curso Análisis de Datos - Informatorio/ventas.csv'
with open(ruta_archivo, 'rb') as f:
  result = chardet.detect(f.read())
  encoding = result['encoding']

# Leer el archivo CSV con el encoding detectado

try:
  df = pd.read_csv(ruta_archivo, encoding=encoding, sep=";", decimal=",")
  print("La extraccion se realizo correctamente")
except FileNotFoundError:
  print(f"El archivo no se encontro en la ruta especificada {ruta_archivo}")
except Exception as e:
  print(f"Ocurrio un error:{e}")

Mounted at /content/drive
La extraccion se realizo correctamente


### 2. Transformación (Transform)

In [3]:
# Muestra las primeras 5 filas del DataFrame
df. head() # 5 es por defecto, pero podemos colocar cualquier valor entre los () y nos va a mostrar esa cantidad de filas



,Tienda,Marca,Tipo,Gï¿½nero,Talla,Color,Categoria,precio de venta,Fecha,Hora
0,Lima,Asics,WB1820,Femenino,42,Azul,Pantalon,89.00,15/07/2015,07:32:00
1,Lima,Asics,Kayano Single Tab,Usinex,42-44,Azul,Ropa interior,24.99,15/07/2015,07:33:00
2,Lima,Asics,WB1820,Femenino,37,Rosado,Pantalon,89,15/07/2015,07:52:00
3,Lima,Asics,WB2585,Femenino,39,Negro,Pantalon,99,15/07/2015,07:58:00
4,Lima,Asics,WB1820,Femenino,46,Multicolor,Pantalon,89,15/07/2015,08:19:00


In [4]:
# Resumen de la información del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Tienda           800 non-null    object
 1   Marca            800 non-null    object
 2   Tipo             800 non-null    object
 3   Gï¿½nero         800 non-null    object
 4   Talla            800 non-null    object
 5   Color            800 non-null    object
 6   Categoria        800 non-null    object
 7   precio de venta  800 non-null    object
 8   Fecha            800 non-null    object
 9   Hora             800 non-null    object
dtypes: object(10)
memory usage: 62.6+ KB


In [5]:
# Renombrar columnas para facilitar ell acceso y evitar caracteres especiales
df.rename(columns = {
    'Tienda' : 'tienda',
    'Marca' : 'marca',
    'Tipo' : 'tipo de producto',
    'Gï¿½nero' : 'genero',
    'Talla' : 'talla',
    'Color' : 'color',
    'Categoria' : 'categoria',
    'precio de venta' : 'precio',
    'Fecha' : 'fecha',
    'Hora' : 'hora'
}, inplace=True) # inplace reemplaza en el mismo df

In [6]:
# Converti la columna de "precio" a numerica
df['precio'] = df['precio'].astype(str) #fuerza que el precio de venta sea un string
df['precio'] = df['precio'].str.replace(',', '.', regex=True).astype(float) #reemplaza la coma por un punto y convierte a flotante

# print(set(df['genero'])) # Devuelve las categorias no repetidas (en este caso, las de genero)

# df['genero'] = df['genero'].str.replace('Masculino', 'M', regex=False)
# df['genero'] = df['genero'].str.replace('Femenino', 'F', regex=False)
# df['genero'] = df['genero'].str.replace('Usinex', 'X', regex=False)
# otra forma de hacerlo es concatenando
df['genero'] = df['genero'].str.replace('Masculino', 'M', regex=False).str.replace('Femenino', 'F', regex=False).str.replace('Usinex', 'X', regex=False)

# Crear una columna nueva a partir de las columnas Fecha y Hora y despues eliminar esas 2 columnas
df['fecha_hora'] = pd.to_datetime(df['fecha'] + ' ' + df['hora'], format='%d/%m/%Y %H:%M:%S')


#df['fecha_hora'] = df['fecha_hora'].dt.strftime('%d/%m/%Y %H:%M:%S')
df.drop(columns=['fecha', 'hora'], inplace=True)


In [7]:
# Dedocraticamente tomamos la primera aparicion numerica y descartamos el resto
# Tomamos el valor de la izqueirda si hay un guion
df['talla'] = df['talla'].astype(str).apply(lambda x: x.split('-')[0] )

# Ahora convertimos a tipo numerico (entero)
df['talla'] = pd.to_numeric(df['talla'], errors='coerce').astype('Int64')

# Eliminar filas con tallas nulas o no numericas despues de la converion
df.dropna(subset=['talla'], inplace=True)


print("Transformación de datos exitosa")

Transformación de datos exitosa


In [9]:
print(df['categoria'].unique())
print('-')
print(df.groupby('categoria').size())
print('-')
print(df.groupby('categoria')['talla'].unique())
print('-')
print(df.groupby('tipo de producto')['talla'].unique())

['Pantalon' 'Ropa interior' 'Zapatillas']
-
categoria
Pantalon         331
Ropa interior    265
Zapatillas       131
dtype: int64
-
categoria
Pantalon         [42, 37, 39, 46, 38, 40, 41, 45, 43, 36, 44]
Ropa interior                        [42, 38, 36, 40, 46, 44]
Zapatillas       [44, 39, 42, 37, 38, 41, 40, 43, 45, 46, 36]
Name: talla, dtype: object
-
tipo de producto
Air Zoom                                           [42, 44, 38, 45, 36, 43, 39, 40]
Boston 5                                       [36, 42, 37, 40, 46, 41, 43, 39, 44]
Cummulus 17                            [44, 38, 39, 40, 41, 46, 43, 37, 45, 36, 42]
Dri-Fit No-Show                                            [40, 42, 46, 44, 38, 36]
Free 5.0                                   [38, 40, 43, 45, 37, 42, 46, 36, 44, 39]
Gel Quantics                                                           [37, 46, 41]
Hera Deux Single Tab                                       [42, 40, 46, 38, 36, 44]
Hyper Elite Crew                      

In [ ]:
# Generar estádisticas descriptivas de las columnas numéricas
df.describe()

# COUNT son la cantidad de registros
# MEAN es el promedio (en este caso precio promedio de venta)
# STD es la desviacion standard, indica que tan disperso (separados) estan los datos.
# MIN valor minimo
# MAX valor maximo
# 25%, 50%, 75% son los Quartiles
# 50% conocido como la media. Suma de todos los valores dividido cantidad de valores

,talla,precio,fecha_hora
count,727.0,727.000000,727
mean,41.0,78.733274,2015-07-18 08:54:12.132049664
min,36.0,12.990000,2015-07-15 07:32:00
25%,38.0,21.000000,2015-07-16 14:08:00
50%,41.0,89.000000,2015-07-17 18:00:00
75%,44.0,99.000000,2015-07-20 08:14:30
max,46.0,239.000000,2015-07-22 23:06:00
std,3.238882,55.471224,NaN


### 3. Carga (Load)

In [ ]:
# Definir la ruta de salida para guardar el nuevo CSV
ruta_salida = '/content/drive/MyDrive/Curso Análisis de Datos - Informatorio/ventas_transformadas.csv'

# Guardar el dataframe transformado en un nuevo archivo CSV
df.to_csv(ruta_salida, index=False, sep=',') # aca creamos el csv y colocamos separaciones por ,

print(f'Archivo CSV guardado en: {ruta_salida}')

Archivo CSV guardado en: /content/drive/MyDrive/Curso Análisis de Datos - Informatorio/ventas_transformadas.csv
